<a href="https://colab.research.google.com/github/amita-rahma-permatasari/CNN/blob/main/UAS_SistemCerdas_Management_Waste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Import Libraries***

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import urllib
import itertools
import random, os, glob
import pickle
from imutils import paths
from sklearn.utils import shuffle
from urllib.request import urlopen
from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, SpatialDropout2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img


# ***Mounting Google Drive***

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***Import Dataset***

In [3]:
base_dir = "/content/drive/MyDrive/Garbage classification/Garbage classification"
target_size = (224, 224)

In [4]:
waste_labels = {"cardboard": 0, "glass": 1, "metal": 2, "paper": 3, "plastic": 4, "trash": 5}

In [5]:
if os.path.exists(base_dir):
    print(f"The directory {base_dir} exists.")

    # List the contents of the directory
    print("Contents of the directory:")
    for item in os.listdir(base_dir):
        print(item)
else:
    print(f"The directory {base_dir} does not exist.")

The directory /content/drive/MyDrive/Garbage classification/Garbage classification exists.
Contents of the directory:
cardboard
glass
paper
plastic
trash
metal


# ***Load Dataset***

In [6]:
def load_dataset(path):
    x = []
    labels = []
    image_paths = sorted(list(paths.list_images(path)))
    for image_path in image_paths:
        img = cv2.imread(image_path)
        img = cv2.resize(img, target_size)
        x.append(img)
        label = image_path.split(os.path.sep)[-2]
        labels.append(waste_labels[label])

    x, labels = shuffle(x, labels, random_state=42)
    input_shape = (np.array(x[0]).shape[0], np.array(x[0]).shape[1], 3)

    print("X shape: ", np.array(x).shape)
    print(f"Number of Labels: {len(np.unique(labels))} , Number of Observations: {len(labels)}")
    print("Input Shape: ", input_shape)

    return np.array(x), np.array(labels), input_shape

In [ ]:
x, labels, input_shape = load_dataset(base_dir)

In [ ]:
x = x / 255.0

# ***Data Preparation***

In [ ]:
def CNN_data_preparation():
  train = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             validation_split=0.1,
                             rescale=1./255,
                             shear_range=0.1,
                             zoom_range=0.1,
                             width_shift_range=0.1,
                             height_shift_range=0.1)
  test = ImageDataGenerator(rescale=1/255, validation_split=0.1)
  train_generator = train.flow_from_directory(directory=base_dir,
                                              target_size=(target_size),
                                              class_mode="categorical",
                                              subset="training")
  test_generator = test.flow_from_directory(directory=base_dir,
                                            target_size=(target_size),
                                            batch_size=251,
                                            class_mode="categorical",
                                            subset="validation")
  return train_generator, test_generator

In [ ]:
train_generator, test_generator = CNN_data_preparation()

In [ ]:
import os

total = 0
# Access the directory path from the DirectoryIterator object
for category in os.listdir(train_generator.directory):
    count= 0
    for image in os.listdir(os.path.join(train_generator.directory, category)):
        count += 1
        total +=1
    print(str(category).title() + ": " + str(count))
print(f"\nTotal number of train images: {total}")

In [ ]:
# Assuming 'train_generator' is defined and accessible from previous cells
class_names = list(train_generator.class_indices.keys())  # Get class names
class_dis = []  # Placeholder for class distribution

# Calculate class distribution
for category in class_names:
    category_path = os.path.join(train_generator.directory, category)
    count = len(os.listdir(category_path))
    class_dis.append(count)

# Create DataFrame
DF = pd.DataFrame({
    'Class names': class_names,
    'Count': class_dis
})

# Plot with color palette
plt.figure(figsize=(12, 9))
palette = sns.color_palette("viridis", len(class_names))  # You can change "viridis" to any other palette
ax = sns.barplot(x='Class names', y='Count', data=DF, palette=palette)
ax.bar_label(ax.containers[0])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

# Add title and labels with explicit font sizes
plt.title('Class Distribution', fontsize=16)
plt.xlabel('Class Names', fontsize=14)
plt.ylabel('Count', fontsize=14)

# Add annotations for each bar
for container in ax.containers:
    ax.bar_label(container, fontsize=12, color='black')

plt.tight_layout()
plt.show()

# ***Visualization Image***

In [ ]:
def visualization_img(image_batch, labels, num_of_img):
  plt.figure(figsize=(10,10))
  for n in range(num_of_img):
    ax = plt.subplot(5,5,n+1)
    plt.imshow(image_batch[n])
    plt.title(np.array(list(waste_labels.keys()))[to_categorical(labels, num_classes=len(np.unique(labels)))[n] == 1][0].title())
    plt.axis("off")

In [ ]:
visualization_img(x, labels, 10)

# ***Modelling***

In [ ]:
def CNN_create_and_fit_model(train_generator, test_generator, summary=True, fit=True, epochs=10):
  model = Sequential()

  model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", input_shape=(input_shape), activation="relu"))
  model.add(MaxPooling2D(pool_size=2, strides=(2,2)))

  model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=2, strides=(2,2)))

  model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=2, strides=(2,2)))

  model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=2, strides=(2,2)))

  model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=2, strides=(2,2)))

  model.add(Flatten())

  model.add(Dense(units=64, activation="relu"))
  model.add(Dropout(rate=0.2))

  model.add(Dense(units=32, activation="relu"))
  model.add(Dropout(rate=0.2))

  model.add(Dense(units=64, activation="relu"))
  model.add(Dropout(rate=0.2))

  model.add(Dense(units=32, activation="relu"))
  model.add(Dropout(rate=0.2))

  model.add(Dense(units=6, activation="softmax"))

  model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), "acc"])

  callbacks = [EarlyStopping(monitor="val_loss", patience=50, verbose=1, mode="min"),
              ModelCheckpoint(filepath="mymodel.h5", monitor="val_loss", mode="min", save_best_only=True, save_weights_only=False, verbose=1)]

  if summary:
    model.summary()

  if fit:
    history = model.fit_generator(generator=train_generator, epochs=epochs, validation_data=test_generator,
                                callbacks=callbacks, workers=4, steps_per_epoch=700, validation_steps=251//32) # steps_per_epoch=2276//32, validation_steps=251//32
                                                                                                              # you can choose 2276/32 is 71 per epoch for best accuracy

  return model, history

In [ ]:
model, history = CNN_create_and_fit_model(train_generator, test_generator)

# ***Evaluation***

In [ ]:
def CNN_model_evaluate(model):
  loss, precision, recall, acc = model.evaluate(test_generator, batch_size=32)
  print("Test Accuracy: %.2f" % (100 * acc))
  print("Test Loss: %.2f" % (100 * loss))
  print("Test Precision: %.2f" % (100 * precision))
  print("Test Recall: %.2f" % (100 * recall))

  X_test, y_test = test_generator.next()
  y_pred = model.predict(X_test)
  y_pred = np.argmax(y_pred, axis=1)
  y_test = np.argmax(y_test, axis=1)
  target_names = list(waste_labels.keys())
  print(classification_report(y_test, y_pred, target_names=target_names))

  plt.figure(figsize=(20,5))
  plt.subplot(1,2,1)
  plt.plot(history.history["acc"], color="r", label="Training Accuracy")
  plt.plot(history.history["val_acc"], color="b", label="Validation Accuracy")
  plt.legend(loc="lower right")
  plt.xlabel("Epoch", fontsize=16)
  plt.ylabel("Accuracy", fontsize=16)
  plt.ylim([min(plt.ylim()),1])
  plt.title("Training and Validation Accuracy", fontsize=16)

  plt.subplot(1,2,2)
  plt.plot(history.history["loss"], color="r", label="Training Loss")
  plt.plot(history.history["val_loss"], color="b", label="Validation Loss")
  plt.legend(loc="upper right")
  plt.xlabel("Epoch", fontsize=16)
  plt.ylabel("Loss", fontsize=16)
  plt.ylim([0, max(plt.ylim())])
  plt.title("Training and Validation Loss", fontsize=16)

  return y_test, y_pred

In [ ]:
y_test, y_pred = CNN_model_evaluate(model)

# ***Confusion Matrix***

In [ ]:
cm = confusion_matrix(y_test, y_pred)
def plot_confusion_matrix(cm, classes, normalize=False, title="Confusion Matrix", cmap=plt.cm.OrRd):
  if normalize:
    cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

  plt.figure(figsize=(8,6))
  plt.imshow(cm, interpolation="nearest", cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)
  fmt = ".2f" if normalize else "d"
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
  plt.tight_layout()
  plt.ylabel("True Labels", fontweight="bold")
  plt.xlabel("Predicted Labels", fontweight="bold")

In [ ]:
plot_confusion_matrix(cm, waste_labels.keys())

# ***Model Testing***

In [ ]:
from keras.preprocessing import image
def CNN_model_testing(path):
  img = image.load_img(path, target_size=(target_size))
  img = image.img_to_array(img, dtype=np.uint8)
  img = np.array(img)/255.0
  p = model.predict(img.reshape(1,224,224,3))
  predicted_class = np.argmax(p[0])
  return img, p, predicted_class

In [ ]:
img, p, predicted_class = CNN_model_testing("/content/drive/MyDrive/Garbage classification/test6.jpg")


In [ ]:
waste_labels = {0:"cardboard", 1:"glass", 2:"metal", 3:"paper", 4:"plastic", 5:"trash"}
def plot_model_testing(img, p, predicted_class):
  plt.axis("off")
  plt.imshow(img.squeeze())
  plt.title("Maximum Probabilty: " + str(np.max(p[0], axis=-1)) + "\n" + "Predicted Class: " + str(waste_labels[predicted_class]))
  plt.imshow(img);

In [ ]:
plot_model_testing(img, p, predicted_class)